In [1]:
# GerryChain is a library for using Markov Chain Monte Carlo methods to study the problem of political redistricting. 
#   Development of the library began during the 2018 Voting Rights Data Institute (VRDI).
#
# Install instructions: https://gerrychain.readthedocs.io/en/latest/user/install.html

from gerrychain import Graph

# Read Oklahoma county graph from the json file "OK_county.json"
filepath = 'districting-data\\'
filename = 'OK_county.json'

# GerryChain has a built-in function for reading graphs of this type:
G = Graph.from_json( filepath + filename )

In [2]:
# For each node, print the node #, county name, and its population
for node in G.nodes:
    county_name = G.nodes[node]['NAME20']
    county_population = G.nodes[node]['P0010001']
    G.nodes[node]['TOTPOP'] = county_population
    print("Node",node,"represents",county_name,"County, which had a population of",county_population,"in the 2020 census.")

Node 0 represents Washita County, which had a population of 10924 in the 2020 census.
Node 1 represents Jackson County, which had a population of 24785 in the 2020 census.
Node 2 represents Major County, which had a population of 7782 in the 2020 census.
Node 3 represents Delaware County, which had a population of 40397 in the 2020 census.
Node 4 represents Custer County, which had a population of 28513 in the 2020 census.
Node 5 represents Ellis County, which had a population of 3749 in the 2020 census.
Node 6 represents Oklahoma County, which had a population of 796292 in the 2020 census.
Node 7 represents Johnston County, which had a population of 10272 in the 2020 census.
Node 8 represents Comanche County, which had a population of 121125 in the 2020 census.
Node 9 represents Pushmataha County, which had a population of 10812 in the 2020 census.
Node 10 represents Cleveland County, which had a population of 295528 in the 2020 census.
Node 11 represents Wagoner County, which had a p

In [3]:
# we are to solve the following task:
# input: a population vector, desired number of districts k
# output: a partition of the populations into k districts (not necessarily connected!) 
#            to minimize the difference between most and least populated districts

import gurobipy as gp
from gurobipy import GRB

In [4]:
k = 5  # desired number of districts

# create model 
m = gp.Model()

# create variables
x = m.addVars( G.nodes, k, vtype=GRB.BINARY )  # x[i,j] = 1 when county i is assigned to district j
y = m.addVar()                                 # y = population of smallest district
z = m.addVar()                                 # z = population of largest district

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-01


In [5]:
# objective is to minimize absolute population deviation
m.setObjective( z - y, GRB.MINIMIZE )

# add constraints saying that each county i is assigned to one district
m.addConstrs( gp.quicksum( x[i,j] for j in range(k) ) == 1 for i in G.nodes ) 

# add constraints saying that each district has population at least y
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes ) >= y for j in range(k) )

# add constraints saying that each district has population at most z
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes ) <= z for j in range(k) )

m.update()

In [6]:
# solve IP model
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 87 rows, 387 columns and 1165 nonzeros
Model fingerprint: 0x4c85ac40
Variable types: 2 continuous, 385 integer (385 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1731757.0000
Presolve time: 0.00s
Presolved: 87 rows, 387 columns, 1165 nonzeros
Variable types: 0 continuous, 387 integer (385 binary)

Root relaxation: objective 0.000000e+00, 119 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    9 1731757.00    0.00000   100%     -    0s
H    0     0                    56060.000000    0.00000   100%     -    0s


In [7]:
# print the absolute population deviation
print("The minimum required deviation is",m.objVal,"persons.")

# retrieve the districts and their populations
districts = [ [ i for i in G.nodes if x[i,j].x > 0.5 ] for j in range(k) ]
district_county_names = [ [ G.nodes[i]['NAME20'] for i in district ] for district in districts ]
district_populations = [ sum( G.nodes[i]['TOTPOP'] for i in district ) for district in districts ]

# print district info
for j in range(k):
    print("District",j,"has these nodes =",districts[j],"and this population =",district_populations[j] )
    print("The corresponding county names are =",district_county_names[j] )
    print("")

The minimum required deviation is 5527.0 persons.
District 0 has these nodes = [0, 4, 20, 22, 34, 40, 54, 66] and this population = 790765
The corresponding county names are = ['Washita', 'Custer', 'Jefferson', 'Okfuskee', 'Tulsa', 'Haskell', 'Ottawa', 'Seminole']

District 1 has these nodes = [6] and this population = 796292
The corresponding county names are = ['Oklahoma']

District 2 has these nodes = [1, 3, 7, 9, 11, 12, 13, 14, 21, 23, 26, 27, 31, 39, 42, 43, 45, 46, 51, 53, 57, 58, 63, 69, 70, 72, 74, 75, 76] and this population = 790765
The corresponding county names are = ['Jackson', 'Delaware', 'Johnston', 'Pushmataha', 'Wagoner', 'Beckham', 'Garvin', 'Craig', 'Pawnee', 'McClain', 'Dewey', 'Choctaw', 'Texas', 'Nowata', 'Payne', 'Cimarron', 'Adair', 'Sequoyah', 'Lincoln', 'Muskogee', 'Mayes', 'Coal', 'Atoka', 'Kay', 'Blaine', 'Creek', 'Kiowa', 'Love', 'Noble']

District 3 has these nodes = [2, 5, 10, 18, 25, 30, 35, 36, 47, 49, 50, 52, 55, 56, 61, 67, 73] and this population = 

In [8]:
import networkx as nx
for district in districts:
    print("Is district =", district, "connected?", nx.is_connected( G.subgraph( district ) ) )

Is district = [0, 4, 20, 22, 34, 40, 54, 66] connected? False
Is district = [6] connected? True
Is district = [1, 3, 7, 9, 11, 12, 13, 14, 21, 23, 26, 27, 31, 39, 42, 43, 45, 46, 51, 53, 57, 58, 63, 69, 70, 72, 74, 75, 76] connected? False
Is district = [2, 5, 10, 18, 25, 30, 35, 36, 47, 49, 50, 52, 55, 56, 61, 67, 73] connected? False
Is district = [8, 15, 16, 17, 19, 24, 28, 29, 32, 33, 37, 38, 41, 44, 48, 59, 60, 62, 64, 65, 68, 71] connected? False
